In [89]:
import requests
import time
import os, json
import csv
import concurrent.futures
# from functools import partial
import pandas as pd

In [90]:
# url = "https://refract.fosfor.com/magiccodermodel/6188a3e2-bc43-444e-bfaa-8159fca71b3a/score"
url = "http://vllm-serving-service:8888/v1/completions"
# lltoken = "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJ4WTdTd3k5UE1xaXRDQmNSMm5qcVl6bmoxS3NqZzV3TmdOV0xDVzdyUkhvIn0.eyJleHAiOjE3MzA4ODM3MzgsImlhdCI6MTY5OTI2MTMzOCwiYXV0aF90aW1lIjoxNjk5MjUyMDQ0LCJqdGkiOiJmN2EzMzQwYy1kNDQwLTRlMzUtYjk2ZS04YzBiMTc0Y2RhODAiLCJpc3MiOiJodHRwczovL3JlZnJhY3QtbG9naW4uZm9zZm9yLmNvbS9hdXRoL3JlYWxtcy9tb3NhaWMiLCJhdWQiOlsibW9zYWljLWdhdGVrZWVwZXIiLCJhY2NvdW50Il0sInN1YiI6IjZjMjU4MWU3LWZmMTItNDljNy04MDJmLWI2ZjQzOWQxZDIwMSIsInR5cCI6IkJlYXJlciIsImF6cCI6Im1vc2FpYy1nYXRla2VlcGVyIiwic2Vzc2lvbl9zdGF0ZSI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJNTE9QUyIsImxvbmdfbGl2ZWRfdG9rZW4iLCJzcGVjdHJhLWRldmVsb3BlciIsImRlZmF1bHQtcm9sZXMtbW9zYWljIiwicmVmcmFjdC1kZXZlbG9wZXIiLCJvZmZsaW5lX2FjY2VzcyIsImFkbWluIiwidW1hX2F1dGhvcml6YXRpb24iLCJyZWZyYWN0LWFkbWluIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19fSwic2NvcGUiOiJvcGVuaWQgZW1haWwgcHJvZmlsZSIsInNpZCI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJuYW1lIjoiUmVmcmFjdCBCRlNJIiwicHJlZmVycmVkX3VzZXJuYW1lIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20iLCJnaXZlbl9uYW1lIjoiUmVmcmFjdCIsImZhbWlseV9uYW1lIjoiQkZTSSIsImVtYWlsIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20ifQ.b6SYLgjo9Veo3GmJ8eZjCTNupQjpfMhzsoXdYjWwRtvRnNjBfx0gOqcugO9OcGn-mm8wwpSGI5uiL30-I6SdWBjsf1ur6GztoX7j-nP_3SrJJn3UhNNqIO8LbsPi5gGRTzWtnfjz92BF1YaCXxQwPY0P_aa8vJ6JxZz5Uctn9aIPIJZZnnjC_GPXtXurmshM_tEN2kwCjhEyr7wYzRqUoMtBGfpLjZREBzgZY-x6JyYiXNtycb1d6PFcCXf7nJVV8ienEC_x7OuciDzfeqd-SQnImvAHH7rqFdi9smBN08AbkDS2uAbMrokHrmbiBpaimrR013VwCWz2KL5QYlWleA"
request_payload_file = '/data/Magicoder_payloads.txt'
report_path = '/data/vLLM_xyzConcurrency_1gpu.csv'
df = pd.DataFrame(columns=['Prompt_No', 'Prompt', 'Response', 'Response_status', 'Time (seconds)'])

In [91]:
payload = {"prompt": "Problem: Create an animated dropdown menu in CSS that appears when a button is clicked.Input: Button element selector, dropdown menu element selector.Output: None (animate the dropdown menu).", 
           "max_tokens": 200,
           "temperature": 0,
           "model": "/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66"}

In [92]:
def read_payloads(file_path):
    with open(file_path, 'r') as file:
        for line in file:
            prompt = json.loads(line.strip())
            yield prompt['payload']

In [93]:
def my_decorator(func):
    def calculate_execution_time(*args, **kwargs):
        global df
        start_time = time.time()
        result, a,b= func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        data = {'Prompt_No':b,'Prompt':a,'Response':result.text, 'Response_status':result.status_code, 'Time (seconds)':f'{execution_time:.6f}'}
        df = df.append(data, ignore_index=True)
        print(f"Execution time: {execution_time:.6f} seconds \n\n")
        return result
    return calculate_execution_time

In [94]:
def save_report(file_path, df):
    print(df)
    df.to_csv(file_path, index=False)

In [95]:
@my_decorator
def inference(prompt_index, payload_data=None):
    headers = {
        'Content-Type': 'application/json'
    }
    
    response = requests.request("POST", url, headers=headers, json=payload_data)
    print(f"Prompt {prompt_index}==> {payload_data['prompt']} \n {response.text}")
    return response, payload_data['prompt'], prompt_index
    
        

In [96]:
def my_worker(thread, payload):
    prompts = read_payloads(request_payload_file)
    try:
        for line, prompt in enumerate(prompts,start=0):
            payload = {**payload, 'prompt': prompt}
            a,b,c = inference(line, payload)
    except Exception as e:
        print(f"Error in thread {thread} - prompt {line}: {e}")

In [97]:
#Model Inference(Request)
if __name__ == "__main__":
    global df
    with concurrent.futures.ThreadPoolExecutor(max_workers=30) as executor:
        futures = [executor.submit(my_worker, _, payload) for _ in range(30)]
        concurrent.futures.wait(futures)
    save_report(report_path, df)

Prompt 0==> Problem: Write a function in python to calculate the factorial of a given non-negative integer.Input: Non-negative integer n. Output: Integer, the factorial of n. 
 {"id":"cmpl-629cdf2687404102a8bd038c739cfdbb","object":"text_completion","created":525587,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\ndef factorial(n):\n    if n == 0:\n        return 1\n    else:\n        return n * factorial(n-1)\n\n# Test the function\nprint(factorial(5))  # Output: 120\nprint(factorial(0))  # Output: 1\nprint(factorial(10))  # Output: 3628800","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":40,"total_tokens":135,"completion_tokens":95}}Prompt 0==> Problem: Write a function in python to calculate the factorial of a given non-negative integer.Input: Non-negative integer n. Output: Integer, the factorial of n. 
 {"id":"cmpl-4a99df817e8946adadcd59de96dae83c","object":"text_comp

In [15]:
# #Model Inference(Request) Same Payload
# if __name__ == "__main__":
#     # Using the generator function to read lines 
#     prompts = list(read_payloads(request_payload_file))
#     index=0
#     global df
#     for i in range(1):
#         print(f"loop {i}+++++++++++++++++++++++++++++")
#         with concurrent.futures.ThreadPoolExecutor(max_workers=30) as executor:
#             futures = [executor.submit(inference, srno, payload) for srno in range(100)]
#             concurrent.futures.wait(futures)
#     save_report(report_path, df)

loop 0+++++++++++++++++++++++++++++
Prompt 1==> Problem: Write a function in python to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams. 
 {"id":"cmpl-f3c65729fed346f39fc4674a9e705cac","object":"text_completion","created":520352,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\nAn anagram is a word or phrase formed by rearranging the letters of a different word or phrase, typically using all the original letters exactly once.\n\ndef are_anagrams(s1, s2):\n    return sorted(s1) == sorted(s2)\n\n# Test the function\nprint(are_anagrams(\"listen\", \"silent\"))  # True\nprint(are_anagrams(\"hello\", \"world\"))  # False","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":45,"total_tokens":148,"completion_tokens":103}}
Execution time: 8.118660 seconds 


Prompt 4==> Problem: Write a function in python to check if two 

Prompt 25==> Problem: Write a function in python to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams. 
 {"id":"cmpl-342c920d2e6f4784ba643e441a2eb74a","object":"text_completion","created":520353,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\nAn anagram is a word or phrase formed by rearranging the letters of a different word or phrase, typically using all the original letters exactly once.\n\ndef are_anagrams(s1, s2):\n    return sorted(s1) == sorted(s2)\n\n# Test the function\nprint(are_anagrams(\"listen\", \"silent\"))  # True\nprint(are_anagrams(\"hello\", \"world\"))  # False","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":45,"total_tokens":148,"completion_tokens":103}}
Prompt 29==> Problem: Write a function in python to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicat

Execution time: 7.728093 seconds 


Execution time: 7.605895 seconds 


Execution time: 8.052507 seconds 


Execution time: 7.545251 seconds 

Execution time: 7.825602 seconds 



Execution time: 8.250600 seconds 


Execution time: 7.640745 seconds 


Execution time: 8.151742 seconds 


Execution time: 7.583045 seconds 

Execution time: 7.704802 seconds 



Prompt 30==> Problem: Write a function in python to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams. 
 {"id":"cmpl-4f203ab7fd1a4bb0ac2a8424fc7c3486","object":"text_completion","created":520360,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\nAn anagram is a word or phrase formed by rearranging the letters of a different word or phrase, typically using all the original letters exactly once.\n\ndef are_anagrams(s1, s2):\n    return sorted(s1) == sorted(s2)\n\n# Test the func


Execution time: 7.941365 seconds 

Execution time: 8.571286 seconds 

Execution time: 8.291329 seconds 



Execution time: 8.533391 seconds 


Execution time: 8.407189 seconds 




Execution time: 8.274467 seconds 


Execution time: 8.718619 seconds 


Execution time: 8.113958 seconds 


Execution time: 8.330332 seconds 


Execution time: 8.357188 seconds 

Execution time: 8.330998 seconds 



Prompt 60==> Problem: Write a function in python to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams. 
 {"id":"cmpl-35bde0b5907648419f139e4988d9a300","object":"text_completion","created":520367,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\nAn anagram is a word or phrase formed by rearranging the letters of a different word or phrase, typically using all the original letters exactly once.\n\ndef are_anagrams(s1, s2):\n    return sorte

Execution time: 8.450169 seconds 


Execution time: 8.299783 seconds 


Execution time: 8.828971 seconds 


Prompt 90==> Problem: Write a function in python to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams. 
 {"id":"cmpl-63c8d8e23f164b1299539102fab91171","object":"text_completion","created":520375,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\nAn anagram is a word or phrase formed by rearranging the letters of a different word or phrase, typically using all the original letters exactly once.\n\ndef are_anagrams(s1, s2):\n    return sorted(s1) == sorted(s2)\n\n# Test the function\nprint(are_anagrams(\"listen\", \"silent\"))  # True\nprint(are_anagrams(\"hello\", \"world\"))  # False","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":45,"total_tokens":148,"completion_tokens":103}}
Execution time: 6.317636 se

In [ ]:
def process_data(data):
    print(f"Processing data: {data}")

def worker():
    # Create a new generator instance for each thread
    gen = my_generator()
    
    try:
        for value in gen:
            process_data(value)
    except Exception as e:
        print(f"Error in thread: {e}")
    finally:
        # Ensure proper cleanup (optional)
        print("Thread cleanup")

# Set the number of threads
num_threads = 3

# Outer loop for reiterating
for _ in range(2):  # Adjust the number of iterations as needed
    # Create a ThreadPoolExecutor with the specified number of threads
    with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
        # Submit tasks to the ThreadPoolExecutor
        futures = [executor.submit(worker) for _ in range(num_threads)]

        # Wait for all tasks to complete
        concurrent.futures.wait(futures)